In [16]:
# !pip install tensorflow==2.4.1
# !pip install transformers
# !pip install pyarrow

In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from math import ceil
from transformers import AlbertTokenizerFast, TFAlbertModel

In [20]:
def check_targets(targs):
    if targs[0] == -1:
        return 1
    else:
        return 0

In [27]:
def create_tfrecords_dataset(data, iter_num, dataset_type='train'):
#     paper_title = tf.keras.preprocessing.sequence.pad_sequences(data['paper_title_tok'].to_list(), 
#                                                                 maxlen=512, dtype='int64', 
#                                                                 padding='post', truncating='post', value=0)
    data['no_target'] = data['target_tok'].apply(check_targets)
    data = data[data['no_target']==0].copy()
    
    paper_title = tf.ragged.constant(data['paper_title_tok'].to_list())
    
    paper_mask = tf.ragged.constant(data['paper_title_mask'].to_list())
    
    targets = tf.keras.preprocessing.sequence.pad_sequences(data['target_tok'].to_list(), maxlen=20, 
                                                            dtype='int64', padding='post', 
                                                            truncating='post', value=0)

    ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(paper_title),
                              tf.data.Dataset.from_tensor_slices(paper_mask),
                              tf.data.Dataset.from_tensor_slices(data['journal_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(data['doc_type_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(targets)))
    
    serialized_features_dataset = ds.map(tf_serialize_example)
    
    filename = f"./iteration_1/tfrecords/{dataset_type}/{str(iter_num).zfill(4)}.tfrecord"
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [22]:
def tf_serialize_example(f0, f1, f2, f3, f4):
    tf_string = tf.py_function(serialize_example, (f0, f1, f2, f3, f4), tf.string)
    return tf.reshape(tf_string, ())

In [23]:
def serialize_example(paper_title, paper_mask, journal, doc_type, targets):
    paper_title_list = tf.train.Int64List(value=paper_title.numpy().tolist())
    paper_mask_list = tf.train.Int64List(value=paper_mask.numpy().tolist())
    journal_list = tf.train.Int64List(value=journal.numpy().tolist())
    doc_type_list = tf.train.Int64List(value=doc_type.numpy().tolist())
    targets_list = tf.train.Int64List(value=targets.numpy().tolist())
    
    paper_title_feature = tf.train.Feature(int64_list = paper_title_list)
    paper_mask_feature = tf.train.Feature(int64_list = paper_mask_list)
    journal_feature = tf.train.Feature(int64_list = journal_list)
    doc_type_feature = tf.train.Feature(int64_list = doc_type_list)
    targets_feature = tf.train.Feature(int64_list = targets_list)
    
    features_for_example = {
        'paper_title': paper_title_feature,
        'paper_mask': paper_mask_feature,
        'journal': journal_feature,
        'doc_type': doc_type_feature,
        'targets': targets_feature
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=features_for_example))
    
    return example_proto.SerializeToString()

In [31]:
def turn_part_file_into_tfrecord(base_path, dataset_type='train'):
    file_list = [x for x in os.listdir(f"{base_path}{dataset_type}") if x.endswith('parquet')]
    file_list.sort()
    print(f"There are {len(file_list)} files for {dataset_type}")
    for i, file_name in enumerate(file_list):
        data = pd.read_parquet(f"{base_path}{dataset_type}/{file_name}")
        print(f"_____File number: {i} ({data.shape[0]} samples)")
        create_tfrecords_dataset(data, i, dataset_type)

In [25]:
base_file_path = f"./iteration_1/tokenized_data/"

#### Without padding

In [32]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'train')

There are 50 files for train
_____File number: 0 (3895117 samples)
_____File number: 1 (5368215 samples)
_____File number: 2 (2782313 samples)
_____File number: 3 (3894716 samples)
_____File number: 4 (3456738 samples)
_____File number: 5 (5367319 samples)
_____File number: 6 (3143801 samples)
_____File number: 7 (3893588 samples)
_____File number: 8 (5366319 samples)
_____File number: 9 (5366313 samples)
_____File number: 10 (5364301 samples)
_____File number: 11 (5358874 samples)
_____File number: 12 (3894438 samples)
_____File number: 13 (5357879 samples)
_____File number: 14 (4450546 samples)
_____File number: 15 (4450080 samples)
_____File number: 16 (3136976 samples)
_____File number: 17 (2225998 samples)
_____File number: 18 (4449857 samples)
_____File number: 19 (4450181 samples)
_____File number: 20 (4452179 samples)
_____File number: 21 (4259843 samples)
_____File number: 22 (4451241 samples)
_____File number: 23 (4251282 samples)
_____File number: 24 (4252101 samples)
_____F

In [28]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'val')

There are 10 files for val
_____File number: 0 (62541 samples)
_____File number: 1 (121255 samples)
_____File number: 2 (120625 samples)
_____File number: 3 (119421 samples)
_____File number: 4 (95079 samples)
_____File number: 5 (61018 samples)
_____File number: 6 (103451 samples)
_____File number: 7 (59970 samples)
_____File number: 8 (90211 samples)
_____File number: 9 (59167 samples)
CPU times: user 5min 51s, sys: 20.1 s, total: 6min 11s
Wall time: 5min 15s


In [29]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'test')

There are 5 files for test
_____File number: 0 (38174 samples)
_____File number: 1 (27355 samples)
_____File number: 2 (29110 samples)
_____File number: 3 (24559 samples)
_____File number: 4 (31432 samples)
CPU times: user 1min, sys: 3.35 s, total: 1min 3s
Wall time: 54.1 s
